## 🎯 Problema a ser resolvido

O objetivo deste MVP é analisar o desempenho dos times do **Campeonato Brasileiro da Série A** ao longo das temporadas de **2003 a 2022**, buscando entender fatores que influenciam os resultados das partidas e comportamentos recorrentes entre os times mais bem-sucedidos (campeões, classificados para a Libertadores) e os de pior desempenho (rebaixados).

A partir dessa análise, queremos extrair insights relevantes que ajudem a responder perguntas como:

- 📌 **O mando de campo ainda tem forte influência nos resultados?**
- 📌 **Os gols marcados tem forte influência nos resultados?**
- 📌 **Como foi o aproveitamento dos times com base no mando de campo e gols marcados/sofridos?**

---

### 📂 Dataset escolhido

Para começar, escolhi no Kaggle o dataset:

**Brazilian League Matches 2003–2022**  
🔗 [Link do dataset no Kaggle](https://www.kaggle.com/datasets/fabrciomacena/brazilian-league-matches-2003-2022)

Este dataset contém informações detalhadas de todas as partidas da Série A entre os anos de 2003 e 2022. Ele inclui:

- Nomes dos times
- Resultados dos jogos
- Datas das partidas
- Locais e estádios

---

### 🔧 Etapa inicial

Vou começar **baixando o dataset e subindo para o Databricks**, para que eu possa manipulá-lo diretamente na plataforma, aplicando transformações, análises e visualizações.


In [0]:
# Leitura do CSV
df = spark.read.option("header", "true").option("inferSchema", "true").csv("/FileStore/tables/jogos_brasileirao.csv")
df.show(5)
df.printSchema()

+-----+--------------+------------+----------+----------+-----------+----+
|round|       stadium|   home_team|home_score|away_score|  away_team|year|
+-----+--------------+------------+----------+----------+-----------+----+
|    1|Brinco de Ouro|     Guarani|       4.0|       2.0|      Vasco|2003|
|    1|   Ligga Arena|Athletico-PR|       2.0|       0.0|     Grêmio|2003|
|    1|Alfredo Jaconi|   Juventude|       2.0|       2.0|  São Paulo|2003|
|    1|      Mineirão|    Cruzeiro|       2.0|       2.0|São Caetano|2003|
|    1|      Maracanã|    Flamengo|       1.0|       1.0|   Coritiba|2003|
+-----+--------------+------------+----------+----------+-----------+----+
only showing top 5 rows

root
 |-- round: integer (nullable = true)
 |-- stadium: string (nullable = true)
 |-- home_team: string (nullable = true)
 |-- home_score: double (nullable = true)
 |-- away_score: double (nullable = true)
 |-- away_team: string (nullable = true)
 |-- year: integer (nullable = true)



Visto que correu tudo bem, vou criar um banco de dados chamado bronze.

In [0]:
%sql

CREATE DATABASE IF NOT EXISTS bronze;

USE bronze;

Vou criar uma tabela chamada jogos e importar os dados do csv para essa tabela.

In [0]:

df = spark.read.option("header", "true").option("inferSchema", "true").csv("/FileStore/tables/jogos_brasileirao.csv")

df.write.format("delta").mode("overwrite").saveAsTable("bronze.jogos")

In [0]:
%sql
SELECT * FROM bronze.jogos LIMIT 5;

round,stadium,home_team,home_score,away_score,away_team,year
1,Brinco de Ouro,Guarani,4.0,2.0,Vasco,2003
1,Ligga Arena,Athletico-PR,2.0,0.0,Grêmio,2003
1,Alfredo Jaconi,Juventude,2.0,2.0,São Paulo,2003
1,Mineirão,Cruzeiro,2.0,2.0,São Caetano,2003
1,Maracanã,Flamengo,1.0,1.0,Coritiba,2003


### Esquema Estrela
Optei por implementar um modelo dimensional em Esquema Estrela, onde os dados são organizados em torno de uma tabela fato (fato_jogos) e diversas tabelas dimensão relacionadas, com o objetivo de facilitar análises por diferentes perspectivas como time, data e local.


A seguir vou criar a tabela silver fazendo algumas transformações:

In [0]:
from pyspark.sql.functions import col, when

from pyspark.sql.functions import col, when

# Leitura da camada bronze
df = spark.table("bronze.jogos")

# Transformações
df_silver = df.withColumnRenamed("home_team", "time_mandante") \
              .withColumnRenamed("away_team", "time_visitante") \
              .withColumnRenamed("home_score", "gols_mandante") \
              .withColumnRenamed("away_score", "gols_visitante") \
              .withColumnRenamed("date", "data_partida") \
              .withColumnRenamed("stadium", "estadio") \
              .withColumn("resultado", when(col("gols_mandante") > col("gols_visitante"), "Mandante")
                                        .when(col("gols_mandante") < col("gols_visitante"), "Visitante")
                                        .otherwise("Empate"))

# Criação de uma view temporária para usar no SQL
df_silver.createOrReplaceTempView("vw_jogos_silver")

In [0]:
%sql
-- Criação do schema silver se ainda não existir
CREATE SCHEMA IF NOT EXISTS silver;

-- Criação da tabela a partir da view temporária
CREATE OR REPLACE TABLE silver.jogos_silver AS
SELECT * FROM vw_jogos_silver;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM silver.jogos_silver LIMIT 5;

round,estadio,time_mandante,gols_mandante,gols_visitante,time_visitante,year,resultado
1,Brinco de Ouro,Guarani,4.0,2.0,Vasco,2003,Mandante
1,Ligga Arena,Athletico-PR,2.0,0.0,Grêmio,2003,Mandante
1,Alfredo Jaconi,Juventude,2.0,2.0,São Paulo,2003,Empate
1,Mineirão,Cruzeiro,2.0,2.0,São Caetano,2003,Empate
1,Maracanã,Flamengo,1.0,1.0,Coritiba,2003,Empate


Agora vou criar a camada gold.

Vou criar uma tabela com estatísticas agregadas por time mandante, por exemplo:

- Total de jogos como mandante

- Total de vitórias, empates, derrotas

- Total de gols marcados e sofridos

- Aproveitamento de pontos (em %)

In [0]:
from pyspark.sql.functions import col, count, sum, when, round

# Leitura da camada Silver
df = spark.table("silver.jogos_silver")

# Cálculo por time mandante
df_gold = df.groupBy("time_mandante").agg(
    count("*").alias("jogos_mandante"),
    sum(when(col("resultado") == "Mandante", 1).otherwise(0)).alias("vitorias"),
    sum(when(col("resultado") == "Empate", 1).otherwise(0)).alias("empates"),
    sum(when(col("resultado") == "Visitante", 1).otherwise(0)).alias("derrotas"),
    sum(col("gols_mandante")).alias("gols_feitos"),
    sum(col("gols_visitante")).alias("gols_sofridos")
).withColumn(
    "aproveitamento", 
    round((col("vitorias") * 3 + col("empates")) / (col("jogos_mandante") * 3) * 100, 2)
)

# Criar uma view temporária para salvar com SQL
df_gold.createOrReplaceTempView("vw_gold_estatisticas_mandante")

In [0]:
%sql
-- Criar schema gold se ainda não existir
CREATE SCHEMA IF NOT EXISTS gold;

-- Criar a tabela gold com os dados agregados
CREATE OR REPLACE TABLE gold.estatisticas_mandante AS
SELECT * FROM vw_gold_estatisticas_mandante;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM gold.estatisticas_mandante LIMIT 5;

time_mandante,jogos_mandante,vitorias,empates,derrotas,gols_feitos,gols_sofridos,aproveitamento
Fortaleza,158,73,41,44,232.0,178.0,54.85
Santos,410,231,106,73,732.0,391.0,64.96
Botafogo,348,162,92,94,532.0,379.0,55.36
Flamengo,409,224,105,80,680.0,383.0,63.33
Atlético-MG,390,218,83,89,677.0,408.0,62.99


## 📚 Catálogo de Dados – Camada Gold (`gold.estatisticas_mandante`)

| Tabela                  | Coluna         | Tipo       | Descrição                                           | Domínio / Faixa Esperada                         |
|------------------------|----------------|------------|-----------------------------------------------------|--------------------------------------------------|
| estatisticas_mandante  | time_mandante  | String     | Nome do time que atuou como mandante                | Nomes oficiais de clubes brasileiros             |
| estatisticas_mandante  | total_jogos    | Integer    | Total de partidas em que o time foi mandante        | Inteiros positivos (> 0)                         |
| estatisticas_mandante  | vitorias       | Integer    | Número de vitórias como mandante                    | 0 até total_jogos                                |
| estatisticas_mandante  | empates        | Integer    | Número de empates como mandante                     | 0 até total_jogos                                |
| estatisticas_mandante  | derrotas       | Integer    | Número de derrotas como mandante                    | 0 até total_jogos                                |
| estatisticas_mandante  | gols_feitos    | Integer    | Total de gols marcados em casa                      | 0 até dezenas (dependendo do número de jogos)    |
| estatisticas_mandante  | gols_sofridos  | Integer    | Total de gols sofridos em casa                      | 0 até dezenas                                    |
| estatisticas_mandante  | saldo_gols     | Integer    | Diferença entre gols feitos e sofridos              | Pode ser negativo, zero ou positivo              |
| estatisticas_mandante  | aproveitamento | Float (%)  | Percentual de aproveitamento em casa                | 0.0 a 100.0                                       |

Agora que nossa base esta pronta, podemos fazer algumas consultas para responder algumas perguntas.

### 1. Quais times mais venceram como mandantes entre 2002 e 2022?

In [0]:
%sql
SELECT time_mandante, vitorias
FROM gold.estatisticas_mandante
ORDER BY vitorias DESC
LIMIT 10;

time_mandante,vitorias
São Paulo,235
Santos,231
Internacional,229
Flamengo,224
Athletico-PR,221
Atlético-MG,218
Grêmio,218
Palmeiras,212
Corinthians,211
Fluminense,206


- Podemos ver que São Paulo, Santos, Internacional, Flamengo e Athletico-PR foram os times com maior número de vitorias jogando em casa.

### 2. Qual time teve o maior aproveitamento como mandante?

In [0]:
%sql
SELECT time_mandante, aproveitamento
FROM gold.estatisticas_mandante
ORDER BY aproveitamento DESC
LIMIT 10;

time_mandante,aproveitamento
Grêmio,66.76
Internacional,66.67
São Paulo,66.34
Athletico-PR,65.38
Santos,64.96
Palmeiras,64.94
Barueri,64.91
Corinthians,63.42
Flamengo,63.33
Atlético-MG,62.99


- Podemos perceber que Grêmio, Internacional, São Paulo, Athletico-PR e Santos são os times com maior aproveitamento como mandante. 
- Mas como é possível o Grêmio ser o time com maior aproveitamento se na análise anterior vimos que o São Paulo é o time com maior vitórias jogando como mandante?
- A resposta é simples, o aproveitamento leva em conta a eficiência e não o número de jogos, os times podem ter a quantidade de jogos diferente.

### 3. Existe correlação entre gols marcados como mandante e aproveitamento?

In [0]:
%sql
SELECT time_mandante, gols_feitos, aproveitamento
FROM gold.estatisticas_mandante;

time_mandante,gols_feitos,aproveitamento
Fortaleza,232.0,54.85
Santos,732.0,64.96
Botafogo,532.0,55.36
Flamengo,680.0,63.33
Atlético-MG,677.0,62.99
Vasco,463.0,53.9
Goiás,478.0,56.27
Internacional,635.0,66.67
Fluminense,630.0,59.27
Ceará,160.0,51.13


- Podemos perceber que geralmente os times que marcam muitos gols, tem um aproveitamento melhor.

### 4. Quais times são mais equilibradas entre ataque e defesa em casa?

In [0]:
%sql
SELECT time_mandante,
       gols_feitos,
       gols_sofridos,
       (gols_feitos - gols_sofridos) AS saldo_gols
FROM gold.estatisticas_mandante
ORDER BY saldo_gols DESC
LIMIT 10;

time_mandante,gols_feitos,gols_sofridos,saldo_gols
Santos,732.0,391.0,341.0
São Paulo,688.0,354.0,334.0
Grêmio,640.0,330.0,310.0
Internacional,635.0,335.0,300.0
Flamengo,680.0,383.0,297.0
Athletico-PR,645.0,357.0,288.0
Palmeiras,641.0,368.0,273.0
Atlético-MG,677.0,408.0,269.0
Cruzeiro,608.0,374.0,234.0
Corinthians,586.0,355.0,231.0


- Neste caso Santos, São Paulo, Grêmio, Internacional e Flamengo tem o maior saldo de gols, o que sugere um equilíbrio entre ataque e defesa.

### 5. Quais times sofrem mais gols como mandantes?

In [0]:
%sql
SELECT time_mandante, gols_sofridos
FROM gold.estatisticas_mandante
ORDER BY gols_sofridos DESC
LIMIT 10;

time_mandante,gols_sofridos
Fluminense,418.0
Atlético-MG,408.0
Santos,391.0
Flamengo,383.0
Botafogo,379.0
Cruzeiro,374.0
Vasco,369.0
Palmeiras,368.0
Athletico-PR,357.0
Corinthians,355.0


- Fluminense, Atlético-MG, Santos, Flamengo e Botafogo indicam fragilidade defensiva mesmo jogando em casa, o que pode ajudar a explicar baixos aproveitamentos, mas nem tanto, visto que o Santos está no Top 5 de melhor aproveitamento, as vezes um time pode tomar muitos gols em casa, mas fazer mais ainda, o que compensa o número de gols tomados.

## 📌 Conclusões Finais

Ao longo deste projeto, foi possível perceber como a **análise de dados no futebol** vai muito além de observar estatísticas isoladas. Os números contam histórias, revelam padrões e trazem insights, mas também mostram que o futebol é um esporte extremamente complexo, onde inúmeros fatores técnicos, táticos, emocionais e até ambientais influenciam diretamente o resultado de uma partida.

Durante a análise, ficou evidente que métricas como **gols marcados**, **gols sofridos** e **aproveitamento** não devem ser vistas de forma desassociada. Um exemplo interessante foi o caso do **Santos**, que mesmo figurando entre os cinco times que mais sofreram gols jogando em casa, também aparece entre os cinco com melhor aproveitamento como mandante. Esse aparente paradoxo mostra que, mesmo com uma defesa mais vulnerável, o clube conseguiu manter alta taxa de vitórias — possivelmente devido a um ataque eficiente ou desempenho pontual em jogos decisivos.

Esse tipo de observação só é possível graças à:

- ✅ Modelagem adequada dos dados  
- ✅ Construção de pipelines robustos de transformação e carga  
- ✅ Formulação de perguntas relevantes com base nos objetivos definidos no início do projeto

Utilizando a **plataforma Databricks**, conseguimos transformar uma base de dados bruta em uma estrutura analítica que nos permitiu explorar diferentes dimensões do jogo.

---

### 🔍 Novas Perguntas Levantadas

Mais do que responder às perguntas levantadas inicialmente, este projeto despertou novas curiosidades:

- ❓ Como se comportam os times fora de casa?  
- ❓ Há padrão de desempenho por década?  
- ❓ Times que marcam mais gols em casa também vencem fora?  
- ❓ Existe uma correlação entre tipo de estádio e desempenho?

Essas e muitas outras questões poderiam ser exploradas com o mesmo dataset ou com a inclusão de **novas fontes de dados** — como escalações, treinadores, presença de torcida, entre outros fatores.

---

### 🎓 Considerações Finais

Foi extremamente enriquecedor ver como os dados podem transformar algo tão emocional como o futebol em um campo fértil para **análise racional** e **tomada de decisões**. A combinação entre a **paixão pelo esporte** e o **poder da análise de dados** abre espaço para projetos ainda mais amplos e completos no futuro.
